<a href="https://colab.research.google.com/github/mariafernanda0011/Classificador-CNH/blob/main/demonstracao_do_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Demonstração de Classificação de Documentos - CNH física e digital**

> Este notebook permite **testar o modelo de classificação de documentos** já treinado.  
Ele carrega o modelo salvo, processa uma imagem enviada pelo usuário e exibe a classe prevista.


### **Faça upload para este notebook dos seguintes arquivos da pasta `processamento` (presentes no repositório):**
[Classificador de CNH](https://github.com/mariafernanda0011/Classificador-CNH.git)

- `extract_features.py`  
- `qr_feature_extractor.py`  
- `color_feature_extractor.py`

Eles são necessários para extrair texto, QR Code e cores da imagem.

### **Você também deve enviar o arquivo do modelo que deseja testar.**
- `xgboost_cnh_classifier.pkl`
- `random_forest_cnh_classifier.plk`
- `naive_bayes_cnh_classifier.plk`

### **Não se esqueça também de anexar a imagem que você quer classificar, altere a descrição da imagem atribuida na etapa 2, que faz o carregamento da imagem.**



In [1]:
# Importando bibliotecas necessárias
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import joblib # Importar joblib para carregar o modelo .pkl
from PIL import Image
import os
import pandas as pd
import xgboost as xgb

#### **1. Carregar o Modelo Salvo**

In [2]:
#model = joblib.load('xgboost_cnh_classifier.pkl')
model = joblib.load('naive_bayes_cnh_classifier.pkl')
#model = joblib.load('random_forest_cnh_classifier.pkl')

#### **2. Carregando a imagem**

In [3]:
img_path = 'nome_da_sua_img.jpg'

# Carregar a imagem
image_to_process = Image.open(img_path)
print(f"Imagem '{img_path}' carregada com sucesso. Formato: {image_to_process.format}, Tamanho: {image_to_process.size}")

Imagem 'cnh_ed.jpg' carregada com sucesso. Formato: JPEG, Tamanho: (1200, 1600)


#### **3. Extraindo features e Testando o modelo**

In [8]:
print("Instalando tesseract-ocr e pytesseract...")
!sudo apt-get update
!sudo apt-get install -y tesseract-ocr
!pip install pytesseract

import pytesseract
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

print("Concluido.")

Instalando tesseract-ocr e pytesseract...
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (so

In [9]:
from extract_features import extrair_texto, contar_palavras, processar_imagem, obter_bag_palavras, gerar_features_bag_palavras, gerar_csv
from qr_feature_extractor import detectar_qr
from color_feature_extractor import get_colors_hsv, generate_color_features

In [10]:
# 1. Extrair texto (OCR) da imagem e conta palavras
resultado = processar_imagem(img_path)
print("Imagem processada.")

# 3. Detectar QR Code
resultado["qr_detectado"] = detectar_qr(img_path)

# 4. Extrair características de cor
hsv_colors = get_colors_hsv(img_path)
color_features_dict = generate_color_features(hsv_colors)
resultado.update(color_features_dict)

df = pd.DataFrame([resultado])

# Gerar Bag of Words Features
df = gerar_features_bag_palavras(df, obter_bag_palavras())

# Correção: Chamar o método .drop() com parênteses e especificar o parâmetro 'columns'
X = df.drop(columns=["nome_arquivo", "texto_extraido"])

#modelo = joblib.load("xgboost_cnh_classifier.pkl")
modelo = joblib.load("naive_bayes_cnh_classifier.pkl")
#modelo = joblib.load("random_forest_cnh_classifier.pkl")

pred = modelo.predict(X)
classe_predita = int(pred[0])

class_labels = {
    0: "CNH digital - aberta",
    1: "CNH digital - frente",
    2: "CNH digital - verso",
    3: "CNH fisica - aberta",
    4: "CNH fisica - frente",
    5: "CNH fisica - verso"
}

print("CLASSIFICAÇÃO FINAL:", class_labels[classe_predita])

Imagem processada.
CLASSIFICAÇÃO FINAL: CNH fisica - aberta


In [ ]:
# Confira visualmente se a classificação foi assertiva
plt.imshow(image_to_process)